In [1]:
# Calculamos el TMY para un lugar, guardando el
# dato del estadístico FS para todas las variables.

import itertools as it

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.interpolate as interp

In [2]:
# Cargamos los archivos.

dir_name = "../data/test/639947_19.41_-99.14_"
columns = [ "Year", "Month", "Day", "Hour", "Minute",
    "Temperature", "Dew Point", "Wind Speed", "GHI" ]

years  = list( range(1998, 2022) )
months = list( range(1, 13) )

# Unimos todos los años.
df = pd.DataFrame( columns = columns )
for y in years:
    df = df.append( pd.read_csv( dir_name + str(y) + ".csv", skiprows = 2 ) )

# Convertimos a fecha.
df["Date"] = pd.to_datetime( df["Year"].astype(str) + "/"
    + df["Month"].astype(str) + "/" + df["Day"].astype(str)
    + " " + df["Hour"].astype(str) + ":00:00" )

# Corregimos formato de columnas.
df = df.drop(columns[0:5], axis = 1).set_index("Date").astype(float)

df

,Temperature,Dew Point,Wind Speed,GHI
Date,,,,
1998-01-01 00:00:00,5.6,2.8,1.4,0.0
1998-01-01 01:00:00,5.3,2.7,1.4,0.0
1998-01-01 02:00:00,5.0,2.4,1.4,0.0
1998-01-01 03:00:00,4.8,2.2,1.4,0.0
1998-01-01 04:00:00,4.5,2.0,1.5,0.0
...,...,...,...,...
2021-12-31 19:00:00,9.2,0.6,1.0,0.0
2021-12-31 20:00:00,8.3,0.9,1.1,0.0
2021-12-31 21:00:00,7.5,1.1,1.1,0.0


In [3]:
# Obtenemos la información diaria.
vnames = [ "GHI", "T_max", "T_min", "T_mean", 
    "Dp_min", "Dp_max", "Dp_mean", "W_max", "W_mean" ]

df_d = df[ [vnames[0]] ].resample("D").sum()
df_d[vnames[1]] = df[ "Temperature" ].resample("D").min()
df_d[vnames[2]] = df[ "Temperature" ].resample("D").max()
df_d[vnames[3]] = df[ "Temperature" ].resample("D").mean()
df_d[vnames[4]] = df[ "Dew Point"   ].resample("D").min()
df_d[vnames[5]] = df[ "Dew Point"   ].resample("D").max()
df_d[vnames[6]] = df[ "Dew Point"   ].resample("D").mean()
df_d[vnames[7]] = df[ "Wind Speed"  ].resample("D").max()
df_d[vnames[8]] = df[ "Wind Speed"  ].resample("D").mean()

df_d

,GHI,T_max,T_min,T_mean,Dp_min,Dp_max,Dp_mean,W_max,W_mean
Date,,,,,,,,,
1998-01-01,4831.0,4.2,20.4,10.816667,-2.1,3.9,1.108333,2.0,1.450000
1998-01-02,5137.0,3.6,21.8,11.800000,-2.6,4.6,1.237500,1.2,0.745833
1998-01-03,5564.0,4.9,22.5,12.837500,-3.0,4.7,1.729167,1.6,0.704167
1998-01-04,5487.0,6.7,23.0,13.525000,-1.8,3.0,0.883333,1.4,0.975000
1998-01-05,5545.0,7.0,23.0,13.650000,-0.4,2.7,1.237500,2.4,1.208333
...,...,...,...,...,...,...,...,...,...
2021-12-27,5369.0,8.7,24.7,15.429167,-0.5,5.8,2.033333,1.6,1.112500
2021-12-28,5257.0,8.5,23.6,14.637500,3.1,6.7,5.166667,2.2,1.362500
2021-12-29,5534.0,7.0,23.6,14.487500,-0.5,6.0,2.350000,2.0,1.200000


In [4]:
# Cálculo del estadístico Finkelstein-Schafer.

# Iteramos para cada variable.
ls_fs = []
ls_t  = []
for v in vnames:

    # Creamos los dataframes que usaremos.
    fs = pd.DataFrame( columns = months, index = years )
    fs_y = fs.copy()
    fs_t = fs.copy()

    # Iteramos para cada mes y cada año.
    for m in months:
        for y in years:
            # Seleccionamos un mes y un año y
            # calculamos su distribución acumulada.
            df_my = df_d.loc[ ( df_d.index.year == y )
                & ( df_d.index.month == m ), [v] ].sort_values( v
                ).reset_index( drop = True )
            df_my.index = ( df_my.index + 1 ) / df_my.shape[0]
            # Seleccionamos un mes para todos los años y 
            # calculamos su distribución acumulada.
            df_m = df_d.loc[ ( df_d.index.month == 1 ), [v]
                ].sort_values( v ).reset_index( drop = True
                ).reset_index().rename( {"index": "CDF_TOT"}, axis = 1 )
            df_m["CDF_TOT"] = ( df_m["CDF_TOT"] + 1 ) / df_m.shape[0]
            # Interpolamos la información para cada año para poder
            # comparar con la información para todos los años.
            df_m["CDF"] = np.interp( df_m[ v ].values,
                df_my[ v ].values, df_my.index )
            # Calculamos el estadístico de Finkelstein-Schafer
            # como la resta de las dos distribuciones acumuladas.
            fs.loc[y, m] = np.abs( df_m["CDF_TOT"] - df_m["CDF"] ).sum()

        # Ordenamos los estadísticos de menor a
        # mayor y anotamos su año correspondiente.
        fs_t[ m ] = fs[ [m] ].values
        fs_y[ m ] = fs[ [m] ].sort_values(m).index.values
        fs[   m ] = fs[ [m] ].sort_values(m).values

    # Reiniciamos los índices y agregamos las tablas a una lista.
    ls_t.append(fs_t)
    fs   =   fs.reset_index(drop = True)
    fs_y = fs_y.reset_index(drop = True)
    ls_fs.append( pd.concat( [fs, fs_y], axis = 0, keys = ["FS", "Year"]
        ).swaplevel(0, 1).sort_index() )

# Juntamos todas las tablas.
df_fs = pd.concat( ls_fs, axis = 0, keys = v )

# Pesos para la suma del estadístico FS.
# Método Sandía.
#weights = np.array( [ [ 12/24, 1/24, 1/24, 2/24, 1/24,
#    1/24, 2/24, 2/24, 2/24 ] ] ).T
# Método NSRDB.
weights = np.array( [ [ 10/24, 1/24, 1/24, 2/24, 1/24,
    1/24, 2/24, 1/24, 1/24 ] ] ).T

# Aplicamos los pesos, umamos el estadístico de cada año
# para todas las variables y lo ordenamos de menor a mayor.
df_t = pd.concat( ls_t,  axis = 0, keys = vnames )
df_t = np.repeat( np.tile(weights, 12), 24, axis = 0 ) * df_t
tot_fs = df_t.groupby( level = 1 ).sum()
tot_y  = tot_fs.copy()
for m in months:
    tot_y[ m ]  = tot_fs[ [m] ].sort_values(m).index.values
    tot_fs[ m ] = tot_fs[ [m] ].sort_values(m).values
tot_fs = tot_fs.reset_index(drop = True)
tot_y  =  tot_y.reset_index(drop = True)
tot = pd.concat( [tot_fs, tot_y], axis = 0, keys = ["FS", "Year"]
        ).swaplevel(0, 1).sort_index()

# Unimos los estadísticos para cada variable
# individual y para todas las variables.
df_fs = pd.concat( [*ls_fs, tot], axis = 0, keys = vnames + ["Total"] )
# Aseguramos que el año sea un número entero.
df_fs.loc[ (slice(None), slice(None), "Year") ] = (
    df_fs.loc[ (slice(None), slice(None), "Year") ].astype(int) )
# Hacemos que el índice sea el orden de los datos de 1 a 24.
df_fs.index = df_fs.index.set_levels( range( 1,
    df_fs.index.get_level_values(1).shape[0] + 1 ), level = 1)

df_fs

1           2           3           4           5   \
GHI   1  FS     15.747931   87.021125  186.967141  259.250334  152.562023   
         Year        2008        2012        2016        2007        2014   
      2  FS     33.537024   97.643531  187.777525  267.801352   216.13702   
         Year        2019        2018        2015        2001        2009   
      3  FS     34.287477  108.298116  243.637736   272.85515  217.471792   
...                   ...         ...         ...         ...         ...   
Total 22 Year        2017        2021        2001        2002        2002   
      23 FS     97.566827  209.057487  221.927691  233.935749  240.610184   
         Year        2010        2013        2011        2011        2007   
      24 FS    103.650485  224.918058  227.967998  234.610075  251.022699   
         Year        1998        1998        1998        2015        2011   

                       6           7           8           9           10  \
GHI   1  FS     64.652852   77.066985   59.928759   51.980775    30.31487   
         Year        2003        2010        2006        2002        2003   
      2  FS    100.477213   93.598894   66.050497   54.058803   52.632317   
         Year        2000        2011        2018        2004        2002   
      3  FS    112.146065  117.618353   79.160217   61.078887    60.19489   
...                   ...         ...         ...         ...         ...   
Total 22 Year        2010        2009        2002        2013        2020   
      23 FS    242.267475  238.071805  208.483259  168.200873  179.446081   
         Year        1999        2000        2015        2008        2012   
      24 FS    248.283863  239.419764  218.860796  187.971574  188.086068   
         Year        1998        2018        2019        1998        2010   

                       11          12  
GHI   1  FS     24.321945   68.818581  
         Year        2011        2010  
      2  FS     29.744203   71.395971  
         Year        2003        1999  
      3  FS     30.792473    94.30882  
...                   ...         ...  
Total 22 Year        2013        2009  
      23 FS    151.911032  131.313608  
         Year        2016        2013  
      24 FS     160.08194   176.15267  
         Year        2019        2015  

[480 rows x 12 columns]

In [199]:
# Evaluación de la persistencia meteorológica.

# Función que compara menor o mayor que dependiendo del caso.
def comp(x):
    if i == 0: return x <= p
    else:      return x >= p

# Iteramos para el caso der percentil 0.33 y 0.66, 
# para cada variable y para cada mes.
#df_run = [ df_d.copy(), df_d.copy() ]
df_run = [ df_d[ ["GHI", "T_mean"] ].copy(), df_d[ ["GHI", "T_mean"] ].copy() ]
for i in range( len(df_run) ):
    #for v in vnames:
    for v in ["GHI", "T_mean"]:
        for m in months:
            # Seleccionamos un mes para todos los años y 
            # calculamos su distribución acumulada.
            df_m = df_d.loc[ df_d.index.month == m, [v]
                ].sort_values( v ).reset_index( drop = True
                ).reset_index().rename( {"index": "CDF_TOT"}, axis = 1 )
            df_m["CDF_TOT"] = ( df_m["CDF_TOT"] + 1 ) / df_m.shape[0]

            # Calculamos el percentil 0.33 o 0.66, según sea el caso.
            p = df_m.loc[ (df_m["CDF_TOT"] <= (i+1)/3 + 1e-10)
                & (df_m["CDF_TOT"] >= (i+1)/3 - 1e-10), v
                ].values[0]

            # Convertimos las corridas que son menores o exceden
            # el percentil en unos y el resto de valores en ceros.
            df_run[i].loc[ df_d.index.month == m, v ] = np.where(
                df_d.loc[ ( df_d.index.month == m ), v ].apply(comp),
                np.ones_like( df_d.loc[ ( df_d.index.month == m ), v ] ),
                np.zeros_like( df_d.loc[ ( df_d.index.month == m ), v ] ) )
            
# Unimos las tablas de los dos percentiles.
df_r = pd.concat( df_run, axis = 1, keys = ["0.33", "0.66"] ).swaplevel(
    0, 1, axis = 1 ).sort_index(axis = 1)

# Creamos una tabla resumen para las estadísticas de las corridas.
a = pd.DataFrame( index = years, columns = months )
b = pd.concat( [a] * 3, axis = 1, keys = ["number", "max", "pass"]
    ).swaplevel(0, 1, axis = 1).sort_index(axis = 1)
c = pd.concat( [b] * 2, axis = 1, keys = ["0.33", "0.66"]
    ).swaplevel(0, 1, axis = 1).sort_index(axis = 1)
df_nr = pd.concat( [c] * len(vnames), axis = 1, keys = vnames )

# Iteramos para cada variable, cada mes, y cada año.   
for p in ["0.33", "0.66"]:         
    #for v in vnames:
    for v in ["GHI", "T_mean"]:
        for m in months:
            for y in years:
                # Seleccionamos un mes y un año.
                a = df_r.loc[ (df_r.index.year == y)
                    & (df_r.index.month == m), (v, p) ]
                # Obtenemos los datos de las corridas
                # de unos y eliminamos los ceros.
                nr = pd.DataFrame( [ (i, len(list(g))) 
                    for i, g in it.groupby(a) ] )
                nr = nr.where( nr.loc[:, 0] == 1, np.nan ).dropna()
                # Encontramos la corrida más larga
                # y contamos la cantidad de corridas.
                df_nr.loc[y, (v, m, p)] = [
                    nr.loc[:, 1].max(), nr.loc[:, 0].sum(), np.nan ]

    # Calculamos los años que pasan para cada percentil.
    df_nr.loc[:, (slice(None), slice(None), p, "pass")] = ~(
        ( (df_nr.max() == df_nr).loc[
        :, (slice(None), slice(None), p, "max") ] ).values
        + ( (df_nr.max() == df_nr).loc[
        :, (slice(None), slice(None), p, "number") ] ).values )

# Calculamos los años que hay que desechar por el criterio de persistencia.
reject = pd.DataFrame( index = years, columns = months )
for m in months:
    reject.loc[:, m] = df_nr.loc[ :,
        ( slice(None), m, slice(None), "pass" ) ].all(axis = 1)
reject = reject.where(reject == False, np.nan)

In [203]:
# Obtenemos la selección final de años para el TMY.

# Creamos la lista de años para cada mes.
df_list = pd.DataFrame(columns = ["Year"], index = range(1, 13))

# Empezamos los años con el menor estadístico FS.
n = 1
# Iteramos para cada mes
for m in range(1, 13):
    # Verificamos si el año pasa o no el criterio de persistencia.
    if not( df_fs.loc[ ("Total", n, "Year"), m ] in reject[m].dropna().index ):
        df_list.loc[m] = df_fs.loc[ ("Total", n, "Year"), m ]

# Iteramos para los siguientes 4 valores de FS.
for n in range(2, 6):
    # Solo iteramos para los meses que no pasaron el
    # criterio de persistencia en el ciclo pasado.
    for m in df_list[ df_list.isnull().any(axis = 1) ].index:
        if not( df_fs.loc[ ("Total", n, "Year"), m ]
            in reject[m].dropna().index ):
            df_list.loc[m] = df_fs.loc[ ("Total", n, "Year"), m ]

df_list

,Year
1,2008
2,2002
3,2007
4,2001
5,2014
6,2000
7,2011
8,2018
9,2004
10,2001


In [204]:
# Construimos el TMY.  

# Empezamos el TMY con el año correspondiente al mes 1.
df_tmy = df[ (df.index.month == df_list.index[0])
    & (df.index.year == df_list.loc[1, "Year"]) ].copy()

# Agregamos el resto de los meses.
for row in df_list.loc[2:].itertuples():
    # Seleccionamos el mes del año correspondiente.
    df_m = df[ (df.index.month == row.Index)
        & (df.index.year == row.Year) ].copy()
    
    # Suavizamos 6 horas con un spline.
    x   = months
    y_1 = ( list( df_tmy.iloc[-6:, -2].values )
        + list( df_m.iloc[0:6, -2].values ) )
    y_2 = ( list( df_tmy.iloc[-6:, -1].values )
        + list( df_m.iloc[0:6, -1].values ) )
    z_1 = interp.splev( x, interp.splrep( x, y_1, s = 1 ) )
    z_2 = interp.splev( x, interp.splrep( x, y_2, s = 1 ) )
    df_tmy.iloc[-6:, -2] = z_1[0:6]
    df_tmy.iloc[-6:, -1] = z_2[0:6]
    df_m.iloc[  0:6, -2] = z_1[6:]
    df_m.iloc[  0:6, -2] = z_2[6:]
    
    # Unimos los meses.
    df_tmy = df_tmy.append( df_m )

# Damos formato a la tabla.
df_tmy = df_tmy.round(decimals = 2)
df_tmy["Month"] = df_tmy.index.month
df_tmy["Day"] = df_tmy.index.day
df_tmy["Hour"] = df_tmy.index.hour
df_tmy = df_tmy[ ["Month", "Day", "Hour"] + columns[5:]
    ].reset_index( drop = True )

df_tmy

,Month,Day,Hour,Temperature,Dew Point,Wind Speed,GHI
0,1,1,0,8.7,6.4,0.5,0.0
1,1,1,1,8.1,6.5,0.5,0.0
2,1,1,2,7.7,6.6,0.6,0.0
3,1,1,3,7.6,6.5,0.6,0.0
4,1,1,4,7.4,6.1,0.7,0.0
...,...,...,...,...,...,...,...
8755,12,31,19,13.3,6.4,0.5,0.0
8756,12,31,20,12.2,6.5,0.6,0.0
8757,12,31,21,11.1,6.7,0.5,0.0
8758,12,31,22,10.6,6.9,0.5,0.0
